In [ ]:
!mamba install pytorch=1.12 torchvision torchaudio cudatoolkit=11.3 -c pytorch -y -q

In [ ]:
!pip install torch-scatter -f https://data.pyg.org/whl/torch-1.12.0+cu113.html
!pip install torch-sparse -f https://data.pyg.org/whl/torch-1.12.0+cu113.html
!pip install torch-geometric

In [ ]:
!mamba install -c conda-forge pyts -q -y

In [ ]:
!pip install llvmpy
!pip install cython
!pip install numba
!pip install pandas
!pip install networkx
!pip install matplotlib
!pip install ts2vg
!pip install pytorch_lightning

In [5]:
import pandas as pd
import numpy as np
import torch
import warnings
import torch.nn.functional as F
import sklearn
import os.path as osp
import matplotlib.pyplot as plt

from torch_geometric.data import Data
from torch_geometric.loader import DataLoader
from pyts.image import MarkovTransitionField
from sklearn.utils import class_weight

from torch.nn import Linear, CrossEntropyLoss
from torch_geometric.nn import global_mean_pool, global_add_pool, global_max_pool, ChebConv, global_sort_pool
from torch_geometric.loader import DataLoader
from torch.nn import Sequential, BatchNorm1d, ReLU, Dropout
from torch_geometric.nn import GCNConv, GINConv, GINEConv, GATv2Conv, GATConv

from tqdm import tqdm
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report
from sklearn.metrics import multilabel_confusion_matrix
from sklearn.metrics import f1_score
from sklearn.manifold import TSNE

from torchvision import transforms
import pytorch_lightning as pl

from pytorch_lightning import Trainer
from pytorch_lightning.loggers import TensorBoardLogger

import ts2vg
from ts2vg import NaturalVG
from ts2vg import HorizontalVG

In [6]:
#function for graph creation
def create_MTF_graph():
    warnings.filterwarnings("ignore")
    # preparation for un/cut graphs
    if len_type == "un/cut":
    
        df = pd.read_csv(path_main)  
        del df['Unnamed: 0']
        df.index, df.columns = [range(df.index.size), range(df.columns.size)]
        length_rss = int((df.columns.stop-2)/2)
        
        X = df.loc[:,df.columns[:length_rss]].to_numpy()
        Y = df[length_rss+1].to_numpy(dtype=np.uint8)
        X_mask = df.loc[:,df.columns[length_rss+2:]].to_numpy()
        
        MTF = MarkovTransitionField(n_bins=length_rss)
        X_gaf = MTF.fit_transform(X)
        
    # preparation for random graphs
    elif len_type == "random":
        dataset_rss = np.load(path_main, allow_pickle=True)['arr_0']
        dataset_properties = np.load(path_properties, allow_pickle=True)['arr_0']
        dataset_mask = np.load(path_mask, allow_pickle=True)['arr_0']

        for i in range(len(dataset_properties)):
            if  dataset_properties[i,1] == True:
                dataset_properties[i,1] = 1
            else:
                dataset_properties[i,1] = 0
        
        X = dataset_rss
        X_mask = dataset_mask
        Y = dataset_properties[:,2]
        Y_len = dataset_properties[:,0]

        X_gaf = []
        for i in range(len(Y_len)):
            
            MTF = MarkovTransitionField(n_bins=Y_len[i])
            X_gaf_temp = MTF.fit_transform(X[i].reshape(1, -1))
            X_gaf.append(X_gaf_temp[0])
    
    # output will have all graphs 
    output = []
    
    # setting class_weights for graph
    global class_weights
    class_weights = torch.tensor(class_weight.compute_class_weight(class_weight='balanced',
                                                                   classes=np.unique(Y),
                                                                   y=Y))
    # function for creating edge index and edge weight for a given MTF matrix
    def adjToEdgidx(adj_mat):
        edge_index = torch.from_numpy(adj_mat).nonzero().t().contiguous()
        row, col = edge_index
        edge_weight = adj_mat[row, col]#adj_mat[row, col]
        return edge_index, edge_weight
    
    for i, j in enumerate(X_gaf):
        edge_index, edge_weight = adjToEdgidx(j)
        #Into Data save node values "x", edge index from adjacency matrix and edge features/attributes, finally labels
        
        if classif == "graph": # for graph classification
            y_mask = torch.tensor(Y[i], dtype=torch.long)      
        elif classif == "node":                 # for node classification 
            y_mask = torch.unsqueeze(torch.tensor(X_mask[i], dtype=torch.double),1)
            
        output.append(Data(x=torch.unsqueeze(torch.tensor(X[i], dtype=torch.double),1), edge_index=edge_index, edge_attr=torch.unsqueeze(torch.tensor(edge_weight, dtype=torch.double),1), y=y_mask))
    
    return output

In [7]:
 #function for graph creation
def create_visual_graph():
    # warnings.filterwarnings("ignore")
    # preparation for un/cut graphs
    if len_type == "un/cut":
    
        df = pd.read_csv(path_main)  
        del df['Unnamed: 0']
        df.index, df.columns = [range(df.index.size), range(df.columns.size)]
        length_rss = int((df.columns.stop-2)/2)
        
        X = df.loc[:,df.columns[:length_rss]].to_numpy()
        Y = df[length_rss+1].to_numpy(dtype=np.uint8)
        X_mask = df.loc[:,df.columns[length_rss+2:]].to_numpy()
        
        
    # preparation for random graphs
    elif len_type == "random":
        
        dataset_rss = np.load(path_main, allow_pickle=True)['arr_0']
        dataset_properties = np.load(path_properties, allow_pickle=True)['arr_0']
        dataset_mask = np.load(path_mask, allow_pickle=True)['arr_0']

        for i in range(len(dataset_properties)):
            if  dataset_properties[i,1] == True:
                dataset_properties[i,1] = 1
            else:
                dataset_properties[i,1] = 0
        
        X = dataset_rss
        X_mask = dataset_mask
        Y = dataset_properties[:,2]
        Y_len = dataset_properties[:,0]

    # output will have all graphs 
    output = []
    
    # setting class_weights for graph
    global class_weights
    class_weights = torch.tensor(class_weight.compute_class_weight(class_weight='balanced',
                                                                   classes=np.unique(Y),
                                                                   y=Y))
    # function for creating edge index and edge weight for a given MTF matrix
    def adjToEdgidx(X_current):
        g = NaturalVG(weighted='distance')
        # g = HorizontalVG(weighted='distance')
        g.build(X_current)

        adj_mat_visual = np.zeros([len(X_current),len(X_current)], dtype='float')
        for i in range(len(g.edges)):
            x, y, q =g.edges[i]
            adj_mat_visual[x,y] = q#/g.weights.max()
            adj_mat_visual[y,x] = q#/g.weights.max()
        
        edge_index = torch.from_numpy(adj_mat_visual).nonzero().t().contiguous()
        row, col = edge_index
        edge_weight = adj_mat_visual[row, col]
        
        return edge_index, edge_weight
    
    for i in range(len(X)):
        edge_index, edge_weight = adjToEdgidx(X[i])
        
        if classif == "graph": # for graph classification
            y_mask = torch.tensor(Y[i], dtype=torch.long)      
        elif classif == "node":                 # for node classification 
            y_mask = torch.unsqueeze(torch.tensor(X_mask[i], dtype=torch.double),1)
        
        output.append(Data(x=torch.unsqueeze(torch.tensor(X[i], dtype=torch.double),1), 
                           edge_index=torch.tensor(edge_index, dtype=torch.int64), 
                           edge_attr=torch.unsqueeze(torch.tensor(edge_weight, dtype=torch.double),1),
                           y=y_mask))
    return output


In [8]:
def create_join_graph():
    
    warnings.filterwarnings("ignore")
    # preparation for un/cut graphs
    if len_type == "un/cut":
    
        df = pd.read_csv(path_main)  
        del df['Unnamed: 0']
        df.index, df.columns = [range(df.index.size), range(df.columns.size)]
        length_rss = int((df.columns.stop-2)/2)
        
        X = df.loc[:,df.columns[:length_rss]].to_numpy()
        Y = df[length_rss+1].to_numpy(dtype=np.uint8)
        X_mask = df.loc[:,df.columns[length_rss+2:]].to_numpy()
        
        MTF = MarkovTransitionField(n_bins=length_rss)
        X_gaf = MTF.fit_transform(X)
        
    # preparation for random graphs
    elif len_type == "random":
        dataset_rss = np.load(path_main, allow_pickle=True)['arr_0']
        dataset_properties = np.load(path_properties, allow_pickle=True)['arr_0']
        dataset_mask = np.load(path_mask, allow_pickle=True)['arr_0']

        for i in range(len(dataset_properties)):
            if  dataset_properties[i,1] == True:
                dataset_properties[i,1] = 1
            else:
                dataset_properties[i,1] = 0
        
        X = dataset_rss
        X_mask = dataset_mask
        Y = dataset_properties[:,2]
        Y_len = dataset_properties[:,0]

        X_gaf = []
        for i in range(len(Y_len)):
            
            MTF = MarkovTransitionField(n_bins=Y_len[i])
            X_gaf_temp = MTF.fit_transform(X[i].reshape(1, -1))
            X_gaf.append(X_gaf_temp[0])
    
    # output will have all graphs 
    output = []
    
    global class_weights
    class_weights = torch.tensor(class_weight.compute_class_weight(class_weight='balanced',
                                                                   classes=np.unique(Y),
                                                                   y=Y))
        
    def adjToEdgidx(adj_mat_MTF,X_current):
        g = NaturalVG(weighted='distance')
        #g = HorizontalVG(weighted='distance')
        g.build(X_current)
        
        #create matrix for visual
        adj_mat_visual = np.zeros([len(adj_mat_MTF),len(adj_mat_MTF)], dtype='float')
        for i in range(len(g.edges)):
            x, y, q =g.edges[i]
            adj_mat_visual[x,y] = q#/g.weights.max()
            adj_mat_visual[y,x] = q#/g.weights.max()
        
        edge_index = torch.from_numpy(adj_mat_MTF).nonzero().t().contiguous()
        
        #join two edge_weight arrays (visual is converted to fit MTF) 
        row, col = edge_index
        edge_weight = np.zeros([len(row),2], dtype='float')
        edge_weight[:,0] = adj_mat_MTF[row, col]
        edge_weight[:,1] = adj_mat_visual[row, col]
        
        # edge_weight = np.reshape(edge_weight,(len(edge_weight),2))
        
        return edge_index, edge_weight
    
    for i, j in enumerate(X_gaf):
        edge_index, edge_weight = adjToEdgidx(j,X[i])
        
        if classif == "graph": # for graph classification
            y_mask = torch.tensor(Y[i], dtype=torch.long)      
        elif classif == "node":                 # for node classification 
            y_mask = torch.unsqueeze(torch.tensor(X_mask[i], dtype=torch.double),1)
            
        #Into Data save node values "x", edge index from adjacency matrix and edge features/attributes, finally labels       
        output.append(Data(x=torch.unsqueeze(torch.tensor(X[i], dtype=torch.double),1), 
                    edge_index=edge_index, 
                    edge_attr=torch.tensor(edge_weight, dtype=torch.double), 
                    y=y_mask))
    return output

In [9]:
## Focal loss definition

In [10]:
import torch
import torch.nn as nn
import torch.nn.functional as F

class FocalLoss(nn.CrossEntropyLoss):
    ''' Focal loss for classification tasks on imbalanced datasets '''

    def __init__(self, gamma=1, alpha=None, ignore_index=-100, reduction='mean'):
        super().__init__(weight=alpha, ignore_index=ignore_index, reduction='none')
        self.reduction = reduction
        self.gamma = gamma

    def forward(self, input_, target):
        cross_entropy = super().forward(input_, target)
        # Temporarily mask out ignore index to '0' for valid gather-indices input.
        # This won't contribute final loss as the cross_entropy contribution
        # for these would be zero.
        target = target * (target != self.ignore_index).long()
        input_prob = torch.gather(F.softmax(input_, 1), 1, target.unsqueeze(1))
        loss = torch.pow(1 - input_prob, self.gamma) * cross_entropy
        
        
        if self.reduction == 'mean':
            return torch.mean(loss) 
        elif self.reduction == 'sum':
            return torch.sum(loss) 
        else:
            return loss

In [11]:
#keeps the last output so we dont have to generate another if all parameters are the same as beffore
global temp_repeat
temp_repeat=['']*6
def generate_output():
    if temp_repeat[0] != graph_type or temp_repeat[1] != classif or temp_repeat[2] != len_type or temp_repeat[3] != path_main or temp_repeat[4] != path_properties or temp_repeat[5] != path_mask:
    
        global output
        if graph_type == "MTF":
            output = create_MTF_graph()
        if graph_type == "visual":
            output = create_visual_graph()
    
    temp_repeat[0] = graph_type 
    temp_repeat[1] = classif
    temp_repeat[2] = len_type
    temp_repeat[3] = path_main
    temp_repeat[4] = path_properties
    temp_repeat[5] = path_mask
    
    return output

In [188]:
from pytorch_lightning import Trainer
from pytorch_lightning.callbacks import Callback
from pytorch_lightning.callbacks import ModelCheckpoint
from pytorch_lightning.callbacks import EarlyStopping
from pytorch_lightning.callbacks import ModelCheckpoint
from pytorch_lightning.callbacks import LearningRateFinder
import tensorflow as tf
import datetime


early_stop = EarlyStopping(monitor='val_acc',patience=10,strict=False,verbose=False, mode='max')
model_chackpoint = ModelCheckpoint(monitor='val_loss', mode='min')
val_checkpoint = ModelCheckpoint(filename="{epoch}-{step}-{val_loss:.1f}", monitor = "val_loss", mode="min")
latest_checkpoint = ModelCheckpoint(filename="latest-{epoch}-{step}", monitor = "step", mode="max",every_n_train_steps = 500,save_top_k = 1)

log_dir = "logs/fit/" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=log_dir, histogram_freq=1)

class FineTuneLearningRateFinder(LearningRateFinder):
    def __init__(self, milestones, *args, **kwargs):
        super().__init__(*args, **kwargs)
        self.milestones = milestones

    def on_fit_start(self, *args, **kwargs):
        return

    def on_train_epoch_start(self, trainer, pl_module):
        if trainer.current_epoch in self.milestones or trainer.current_epoch == 0:
            self.lr_find(trainer, pl_module)

class LitAutoEncoder(pl.LightningModule):
    def __init__(self):
        super(LitAutoEncoder, self).__init__()
        edge_dim = 1
        dim_h = 32
        
        self.conv1 = GINEConv(
            Sequential(Linear(dim_h, dim_h),
                       BatchNorm1d(dim_h), ReLU(),
                       Linear(dim_h, dim_h), ReLU()), edge_dim=edge_dim)
        
        self.conv2 = GINEConv(
            Sequential(Linear(dim_h, dim_h), BatchNorm1d(dim_h), ReLU(),
                       Linear(dim_h, dim_h), ReLU()), edge_dim=edge_dim)
        
        self.conv3 = GINEConv(
            Sequential(Linear(dim_h, dim_h), BatchNorm1d(dim_h), ReLU(),
                       Linear(dim_h, dim_h), ReLU()), edge_dim=edge_dim)
        
        self.conv4 = GINEConv(
            Sequential(Linear(dim_h, dim_h), BatchNorm1d(dim_h), ReLU(),
                       Linear(dim_h, dim_h), ReLU()), edge_dim=edge_dim)
        
        self.conv5 = GINEConv(
            Sequential(Linear(dim_h, dim_h), BatchNorm1d(dim_h), ReLU(),
                       Linear(dim_h, dim_h), ReLU()), edge_dim=edge_dim)
        
        self.lin1 = Linear(dim_h*5, dim_h*5)
        self.lin2 = Linear(dim_h*5, 5)
        
    def forward(self, data):
        x, edge_index, edge_weight, batch = data.x, data.edge_index, data.edge_attr, data.batch
        
        # Node embeddings 
        h1 = self.conv1(x, edge_index, edge_attr=edge_weight)
        h2 = self.conv2(h1, edge_index, edge_attr=edge_weight)
        h3 = self.conv3(h2, edge_index, edge_attr=edge_weight)
        h4 = self.conv4(h3, edge_index, edge_attr=edge_weight)
        h5 = self.conv5(h4, edge_index, edge_attr=edge_weight)
        
        # Graph-level readout
        
        h1 = global_max_pool(h1, batch)
        h2 = global_max_pool(h2, batch)
        h3 = global_max_pool(h3, batch)
        h4 = global_max_pool(h4, batch)
        h5 = global_max_pool(h5, batch)
        

        # Concatenate graph embeddings
        h = torch.cat((h1, h2, h3, h4, h5), dim=1)

        # Classifier
        h = self.lin1(h)
        h = h.relu()
        h = F.dropout(h, p=0.5, training=self.training)
        h = self.lin2(h)
        
        return h
    
    def configure_optimizers(self):
        optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate, weight_decay=5e-4)
        return optimizer
    
    def training_step(self, train_batch, batch_idx):
                     
        out = model(train_batch)
        loss_function = CrossEntropyLoss(weight=class_weights).to(device) #weight=class_weights
        
        train_loss = loss_function(out, train_batch.y)
        
        correct=out.argmax(dim=1).eq(train_batch.y).sum().item()
        logs={"train_loss": train_loss}
        total=len(train_batch.y)
        
        batch_dictionary={"loss": train_loss, "log": logs, "correct": correct, "total": total}
        
        return train_loss
    
    
    def validation_step(self, val_batch, batch_idx):
      
        out = model(val_batch)
        loss_function = CrossEntropyLoss(weight=class_weights).to(device)
        val_loss = loss_function(out, val_batch.y)
        
        pred = out.argmax(-1)
        correct=out.argmax(dim=1).eq(val_batch.y).sum().item()
        total=len(val_batch.y)
        val_label = val_batch.y
        accuracy = (pred == val_label).sum() / pred.shape[0]
        
        logs={"val_loss": val_loss}
        batch_dictionary={"loss": val_loss, "log": logs, "correct": correct, "total": total}
        self.log("val_loss", val_loss)
        self.log("val_acc", accuracy)
        
    
    def test_step(self, test_batch, batch_idx):
        # this is the test loop
        out = model(test_batch)
        # class_weights.type_as("gpu")
        loss_function = CrossEntropyLoss(weight=class_weights).to(device)
        test_loss = loss_function(out, test_batch.y)
        
        pred = out.argmax(-1)
        test_label = test_batch.y
        accuracy = (pred == test_label).sum() / pred.shape[0]
        self.log("test_loss", test_label)
        self.log("test_acc", accuracy)
        
        
#------------------------------------------------------------
graph_type = "visual"#"visual", "MTF"
classif = "graph" #"graph", "node", set the type of classification
len_type = "un/cut" #"un/cut", "random", set the shape of data used in later paths 
path_main = "dataset_uncut.csv" # "dataset_cut.csv", "dataset_rss.npz", paths used for cut/uncut/random dataset
path_properties = "dataset_properties.npz"  # path to properties used for random 
path_mask = "dataset_mask.npz" # path to mask dataset used for random

# batch_size = 64*2 #set the train batch size
learning_rate = 0.01
range_epoch = 1000 #set length of epoch
epoch_min = 5
epoch_interval = 5

analysis = True
#-------------------------------------------------------------

# my data        
output = generate_output()
torch.manual_seed(6406)

train_size = int(0.6 * len(output))
Temp_size = len(output) - train_size
val_size = int(0.5*Temp_size)
test_size = Temp_size - val_size
train_dataset, val_dataset, test_dataset = torch.utils.data.random_split(output, [train_size, val_size, test_size])
    
train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=64, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=1, shuffle=True)


# setting device
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
        
# model
model = LitAutoEncoder().double()#.to(device)

#training
val_check_interval=len(train_loader)

# trainer = pl.Trainer(max_epochs = 100,
#                      callbacks=[latest_checkpoint,val_checkpoint],
#                      accelerator='gpu',
#                      devices=1,
#                      default_root_dir="saves")

# trainer.fit(model, train_loader, val_loader)

trainer = pl.Trainer(max_epochs = 100,accelerator='gpu',devices=1,default_root_dir="saves")
trainer.fit(model, train_loader, val_loader)

GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name  | Type     | Params
-----------------------------------
0 | conv1 | GINEConv | 2.2 K 
1 | conv2 | GINEConv | 2.2 K 
2 | conv3 | GINEConv | 2.2 K 
3 | conv4 | GINEConv | 2.2 K 
4 | conv5 | GINEConv | 2.2 K 
5 | lin1  | Linear   | 25.8 K
6 | lin2  | Linear   | 805   
-----------------------------------
37.8 K    Trainable params
0         Non-trainable params
37.8 K    Total params
0.151     Total estimated model params size (MB)


Sanity Checking: 0it [00:00, ?it/s]

/opt/conda/lib/python3.9/site-packages/pytorch_lightning/trainer/connectors/data_connector.py:224: PossibleUserWarning: The dataloader, train_dataloader, does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` (try 128 which is the number of cpus on this machine) in the `DataLoader` init to improve performance.
  rank_zero_warn(


Training: 0it [00:00, ?it/s]

Finding best initial lr:   0%|          | 0/100 [00:00<?, ?it/s]

`Trainer.fit` stopped: `max_steps=100` reached.
Restoring states from the checkpoint path at saves/.lr_find_5a770113-0a8a-4e19-982b-5f242854ec7e.ckpt
Restored all states from the checkpoint file at saves/.lr_find_5a770113-0a8a-4e19-982b-5f242854ec7e.ckpt


Validation: 0it [00:00, ?it/s]

/opt/conda/lib/python3.9/site-packages/pytorch_lightning/trainer/call.py:48: UserWarning: Detected KeyboardInterrupt, attempting graceful shutdown...
  rank_zero_warn("Detected KeyboardInterrupt, attempting graceful shutdown...")


In [151]:
trainer.test(model, test_loader)

LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


Testing: 0it [00:00, ?it/s]

────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
       Test metric             DataLoader 0
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
        test_acc            0.9335845708847046
        test_loss           1.9759774208068848
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────


[{'test_loss': 1.9759774208068848, 'test_acc': 0.9335845708847046}]

In [165]:
model = LitAutoEncoder.load_from_checkpoint("saves/lightning_logs/with_LRfinder_e_100/checkpoints/epoch=94-step=9300-val_loss=0.0.ckpt").double()
trainer.test(model, test_loader)

LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


Testing: 0it [00:00, ?it/s]

────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
       Test metric             DataLoader 0
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
        test_acc             0.979745626449585
        test_loss           1.9759774208068848
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────


[{'test_loss': 1.9759774208068848, 'test_acc': 0.979745626449585}]

In [166]:
model = LitAutoEncoder.load_from_checkpoint("saves/lightning_logs/e_100/checkpoints/epoch=99-step=10000-val_loss=0.0.ckpt").double()
trainer.test(model, test_loader)

LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


Testing: 0it [00:00, ?it/s]

────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
       Test metric             DataLoader 0
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
        test_acc             0.97833251953125
        test_loss           1.9759774208068848
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────


[{'test_loss': 1.9759774208068848, 'test_acc': 0.97833251953125}]

In [190]:
tensorboard --logdir=YOUR_LOG_DIR --host=127.0.0.1

In [186]:
%load_ext tensorboard
%tensorboard --logdir lightning_logs/

The tensorboard extension is already loaded. To reload it, use:
  %reload_ext tensorboard
